In [4]:
import modin.pandas as pd

In [5]:
dataset=pd.read_csv('milknew.csv')

In [6]:
dataset.describe()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour
count,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000,1059.000000
mean,6.630123,44.226629,0.546742,0.432483,0.671388,0.491029,251.840415
std,1.399679,10.098364,0.498046,0.495655,0.469930,0.500156,4.307424
min,3.000000,34.000000,0.000000,0.000000,0.000000,0.000000,240.000000
25%,6.500000,38.000000,0.000000,0.000000,0.000000,0.000000,250.000000
50%,6.700000,41.000000,1.000000,0.000000,1.000000,0.000000,255.000000
75%,6.800000,45.000000,1.000000,1.000000,1.000000,1.000000,255.000000
max,9.500000,90.000000,1.000000,1.000000,1.000000,1.000000,255.000000


In [7]:
dataset.head()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
0,6.6,35,1,0,1,0,254,high
1,6.6,36,0,1,0,1,253,high
2,8.5,70,1,1,1,1,246,low
3,9.5,34,1,1,0,1,255,low
4,6.6,37,0,0,0,0,255,medium


In [8]:
dataset.isna().sum()

pH            0
Temprature    0
Taste         0
Odor          0
Fat           0
Turbidity     0
Colour        0
Grade         0
dtype: int64

In [9]:
dataset['Grade'].replace({'High':1,"Medium":0.5,"Low":0},inplace=True)

In [10]:
X=dataset.drop("Grade",axis=1)

In [11]:
Y=dataset["Grade"]

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=10)

In [13]:
print(X.shape, Y.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1059, 7) (1059,) (847, 7) (212, 7) (847,) (212,)


In [16]:

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [23]:
model=DecisionTreeClassifier()
model.fit(x_train,y_train)
prediction=model.predict(x_test[1:212])
print(prediction)

['low' 'low' 'medium' 'low' 'medium' 'high' 'medium' 'low' 'high' 'high'
 'low' 'high' 'low' 'medium' 'medium' 'low' 'medium' 'high' 'medium'
 'medium' 'medium' 'medium' 'low' 'medium' 'medium' 'medium' 'medium'
 'low' 'low' 'low' 'low' 'medium' 'medium' 'high' 'medium' 'medium' 'high'
 'high' 'low' 'medium' 'high' 'medium' 'high' 'high' 'low' 'medium' 'low'
 'low' 'high' 'low' 'high' 'high' 'high' 'high' 'medium' 'low' 'low'
 'medium' 'high' 'high' 'high' 'low' 'high' 'low' 'low' 'low' 'high'
 'medium' 'medium' 'high' 'low' 'high' 'low' 'high' 'low' 'medium' 'high'
 'low' 'low' 'high' 'low' 'high' 'high' 'low' 'medium' 'low' 'low' 'low'
 'medium' 'low' 'medium' 'low' 'high' 'medium' 'high' 'medium' 'high'
 'low' 'low' 'high' 'medium' 'low' 'high' 'low' 'low' 'medium' 'low'
 'high' 'medium' 'medium' 'low' 'low' 'medium' 'medium' 'high' 'low'
 'high' 'medium' 'low' 'medium' 'medium' 'medium' 'low' 'high' 'medium'
 'high' 'low' 'high' 'medium' 'low' 'low' 'high' 'medium' 'low' 'low'
 'lo

In [25]:
metrics.accuracy_score(prediction,y_test[1:212])

0.985781990521327